In [5]:
from tensorflow.keras.models import load_model
import os

%load_ext autoreload 
%autoreload 2

import tensorflow as tf
from tensorflow.keras import layers, models


# From model script, workaround to serialize embed layer
@tf.keras.utils.register_keras_serializable()
class L2Normalization(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.math.l2_normalize(inputs, axis=1)


# Load the trained encoder
encoder = load_model("../gesture_encoder_model.keras", safe_mode=False)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from raw_to_dataset import load_gesture_xyz
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from scipy.spatial.distance import cdist
from raw_to_dataset import sample_points_fixed, load_gesture_xyz

# Computes the distance matrix for a list of points
def compute_distance_matrix(points):
    if (len(points) != 128):
        points = sample_points_fixed(points, N=128)

    """
    points: (num_points, 3)
    returns: (num_points, num_points) distance matrix
    """
    points = np.asarray(points, dtype=np.float32)
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    dist_matrix = np.linalg.norm(diff, axis=-1)
    return dist_matrix

# Gets an embedding for a gesture (represented by a list of xyz points)
def get_embedding(gesture_points):
    """
    gesture_points: (128, 3) numpy array of a gesture
    returns: (EMBED_DIM,) embedding
    """
    dist_matrix = compute_distance_matrix(gesture_points)
    dist_matrix = np.expand_dims(dist_matrix, axis=0)  # batch dimension
    embedding = encoder.predict(dist_matrix, verbose=0)
    return embedding[0]



# Gets a comparison score between 2 gestures
def compare_gestures(gesture1, gesture2):
    """
    gesture1: (128, 3) numpy array of a gesture
    gesture2: (128, 3) numpy array of a gesture
    """
    emb1 = get_embedding(gesture1)
    emb2 = get_embedding(gesture2)

    score = np.linalg.norm(emb1 - emb2)
    return score

# Gets comparison score between 2 gestures, given filepaths to .json representations
def compare_gestures_filepath(filepath1, filepath2):
    """
    filepath1, filepath2: string filepaths to .json format of gestures, can be of any size. See sample data.
    """
    points1 = load_gesture_xyz(filepath1)
    points2 = load_gesture_xyz(filepath2)
    return compare_gestures(points1, points2)



In [7]:
# Change paths to view score
gest_filepath = "sample_data/concert_sample/concert_sample_default.json"

apt_verticalL_score = compare_gestures_filepath(gest_filepath, "sample_data/apt_verticalL_sample/apt_verticalL_sample_default2.json")
apt1_score = compare_gestures_filepath(gest_filepath, "sample_data/apt1_sample/apt1_sample_1.json")
concert_score = compare_gestures_filepath(gest_filepath, "sample_data/concert_sample/concert_sample_default2.json")
house1_score = compare_gestures_filepath(gest_filepath, "sample_data/house1_sample/house1_sample_default2.json")

print(f"apt_verticalL score: {apt_verticalL_score:.4f}")
print(f"apt1 score: {apt1_score:.4f}")
print(f"concert score: {concert_score:.4f}")
print(f"house1 score: {house1_score:.4f}")


apt_verticalL score: 0.3138
apt1 score: 0.4852
concert score: 0.2145
house1 score: 0.6727


In [ ]:
# Compare to all .json gestures in sample_data, 
#  - allows multiple representations of each gesture and takes the average score
base_dir = "sample_data"
reference_filepath = "sample_data/concert_sample/concert_sample_default.json"
# reference_filepath = "sample_data/apt_verticalL_sample/apt_verticalL_sample_default3.json"

avg_scores = {} # Dictionary to store average score for each gesture

# Find average of comparisons to each gesture class' examples
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)

    if not os.path.isdir(folder_path):
        continue

    scores = []

    # Get score of each gesture
    for filename in os.listdir(folder_path):
        if not filename.endswith(".json"):
            continue

        gesture_path = os.path.join(folder_path, filename)

        # Skip the reference file itself in case using sample data as example
        if gesture_path == reference_filepath:
            continue

        dist = compare_gestures_filepath(reference_filepath, gesture_path)
        scores.append(dist)

    if scores:
        avg_scores[folder] = sum(scores) / len(scores)
    else:
        avg_scores[folder] = None  # no valid files

# Print results
for folder, avg in avg_scores.items():
    if avg is None:
        print(f"{folder}: (no gestures found)")
    else:
        print(f"{folder}: {avg:.4f}")

apt1_sample: 0.5412
apt_verticalL_sample: 0.2757
concert_sample: 0.5281
house1_sample: 1.0279
